# Bagpack Prediction Challenge

### Import Necessary Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.model_selection import train_test_split

### Import Data

In [3]:
# Import training data
train_df = pd.read_csv("train.csv")

In [4]:
# Show 1st 5 rows of the data
train_df.head()

,id,Brand,Material,Size,Compartments,Laptop Compartment,Waterproof,Style,Color,Weight Capacity (kg),Price
0,0,Jansport,Leather,Medium,7.0,Yes,No,Tote,Black,11.611723,112.15875
1,1,Jansport,Canvas,Small,10.0,Yes,Yes,Messenger,Green,27.078537,68.88056
2,2,Under Armour,Leather,Small,2.0,Yes,No,Messenger,Red,16.643760,39.17320
3,3,Nike,Nylon,Small,8.0,Yes,No,Messenger,Green,12.937220,80.60793
4,4,Adidas,Canvas,Medium,1.0,Yes,Yes,Messenger,Green,17.749338,86.02312


### General Data Analysis

In [5]:
# Data Description
print(f"No. of observations:{train_df.shape[0]}")
print(f"No. of features:{train_df.shape[1]}")

No. of observations:300000
No. of features:11


In [9]:
# Feature Description
print(train_df.dtypes)
num_cols = [col for col in train_df.columns if train_df[col].dtype != "object"]
cat_cols = [col for col in train_df.columns if train_df[col].dtype == "object"]
print("-" * 60)
print(f"Number of Numeric Features: {len(num_cols)}")
print(num_cols)
print("-" * 60)
print(f"Number of Categorical Features: {len(cat_cols)}")
print(cat_cols)

id                        int64
Brand                    object
Material                 object
Size                     object
Compartments            float64
Laptop Compartment       object
Waterproof               object
Style                    object
Color                    object
Weight Capacity (kg)    float64
Price                   float64
dtype: object
------------------------------------------------------------
Number of Numeric Features: 4
['id', 'Compartments', 'Weight Capacity (kg)', 'Price']
------------------------------------------------------------
Number of Categorical Features: 7
['Brand', 'Material', 'Size', 'Laptop Compartment', 'Waterproof', 'Style', 'Color']


In [11]:
# Missing Value Detection
if train_df.isnull().sum().sum() > 0:
    print(train_df.isnull().sum())
else:
    print("No missing value present in data")

id                         0
Brand                   9705
Material                8347
Size                    6595
Compartments               0
Laptop Compartment      7444
Waterproof              7050
Style                   7970
Color                   9950
Weight Capacity (kg)     138
Price                      0
dtype: int64


In [12]:
# Duplicate row Detection
print(f"Number of duplicate rows: {train_df.duplicated().sum()}")

Number of duplicate rows: 0
